## Final Project

https://ieeexplore.ieee.org/abstract/document/5611564
$$\begin{align}
\dfrac{\Delta\rho}{\Delta T}&=\alpha\rho
\end{align}$$
I'm going to extend this to a continuum:
$$\begin{align}
\dfrac{d\rho}{dT}&=\alpha\rho
\end{align}$$
So we can find that:
$$\rho(T)=\rho(T_0)e^{\alpha(T-T_0)}$$
This is a simplified model, but correctly accounts for the exponential behavior instead of just being a linear relationship (which fails at higher temperatures).

### Case 1: Constrained Wires
We'll assume that the wires will not expand in any direction.
This means that $l$, $A_s$, and $A_\perp$ doesn't change with temperature.

#### Resistivity
$$\begin{align}
\rho(T)&=\rho_0e^{\alpha_\rho(T-T_0)}
\end{align}$$

#### Resistance
$$\begin{align}
R&=\dfrac{l}{A_\perp}\rho(T)\\
 &=\dfrac{\rho_0 l}{A_\perp}e^{\alpha_\rho(T-T_0)}\\
R&=R_0e^{\alpha_\rho(T-T_0)}
\end{align}$$

#### Conservation of Energy
$$\begin{align}
A_s\sigma\epsilon(T^4-T_\text{ambient}^4)&=I^2R\\
A_s\sigma\epsilon(T^4-T_\text{ambient}^4)&=I^2R_0e^{\alpha_\rho(T-T_0)}\\
A_s\sigma\epsilon(T^4-T_\text{ambient}^4)&=I^2R_0e^{\alpha_\rho T}e^{-\alpha_\rho T_0}\\
(T^4-T_\text{ambient}^4)e^{-\alpha_\rho T}&=\dfrac{I^2R_0}{A_s\sigma\epsilon}e^{-\alpha_\rho T_0}\\
\end{align}$$

### Case 2: Unrestricted Wires
We'll assume that the wires will not expand only in the length.
This means that $l$ doesn't change with temperature.

#### Temperature-Dependent Geometry
$$\begin{align}
A_\perp(T)=A_\perp^0e^{2\alpha_M(T-T_0)}\\
A_s(T)=A_s^0e^{\alpha_M(T-T_0)}
\end{align}$$

#### Resistivity
$$\begin{align}
\rho(T)&=\rho_0e^{\alpha_\rho(T-T_0)}
\end{align}$$

#### Resistance
$$\begin{align}
R&=l\dfrac{\rho(T)}{A_\perp(T)}\\
 &=l\dfrac{\rho_0e^{\alpha_\rho(T-T_0)}}{A^0_\perp e^{2\alpha_M(T-T_0)}}\\
 &=l\dfrac{\rho_0}{A^0_\perp}e^{(\alpha_\rho-2\alpha_M)(T-T_0)}\\
R&=R_0e^{(\alpha_\rho-2\alpha_M)(T-T_0)}\\
\end{align}$$

#### Conservation of Energy
$$\begin{align}
A_s\sigma\epsilon(T^4-T_\text{ambient}^4)&=I^2R\\
A_s^0e^{\alpha_M(T-T_0)}\sigma\epsilon(T^4-T_\text{ambient}^4)&=I^2R_0e^{(\alpha_\rho-2\alpha_M)(T-T_0)}\\
(T^4-T_\text{ambient}^4)e^{(3\alpha_M-\alpha_\rho)T}&=\dfrac{I^2R_0}{A_s^0\sigma\epsilon}e^{(3\alpha_M-\alpha_\rho)T_0}\\
(T^4-T_\text{ambient}^4)e^{\kappa T}&=\dfrac{I^2R_0}{A_s^0\sigma\epsilon}e^{\kappa T_0}\\
(\kappa T)^4e^{\kappa T}&=\dfrac{(3\alpha_M-\alpha_\rho)^4 R_0}{A_s^0\sigma\epsilon}e^{\kappa T_0}I^2
\end{align}$$

#### Explicit Solution
$$\begin{align}
(\kappa T)^4e^{\kappa T}&=\dfrac{(3\alpha_M-\alpha_\rho)^4 R_0}{A_s^0\sigma\epsilon}e^{\kappa T_0}I^2\\
T(I)&=\dfrac{4}{\kappa}W\left(\sqrt{I}\sqrt[4]{\dfrac{(3\alpha_M-\alpha_\rho)^4 R_0}{256A_s^0\sigma\epsilon}e^{\kappa T_0}}\right)
\end{align}$$

Where $W$ is the Lambert W Function.






### Copper Material
$\rho_0$ at $20^\circ$C : $1.68\cdot10^{−8}$ Ohm meters
https://www.thoughtco.com/table-of-electrical-resistivity-conductivity-608499



I will be starting with the Stefan-Boltzmann law:
$$\begin{align}
P=A\epsilon\sigma T^4
\end{align}$$
Where $P$ is power, $\epsilon$ is the emissivity where $0\le\epsilon\le1$, $\sigma$ is the Stefan-Boltzman constant, and $T$ is the temperature.

We will be modeling a wire in a vacuum and the equalibrium temperature where the input current leads to a temperature of the wire. We assume that the wire is not going to melt, that all energy loss is through radiation, and that the length of the wire will not increase (constrained at the ends).



